### Submitted Version

In [43]:
import numpy as np

# define precision to ensure at least 10 digit precision
np.set_printoptions(precision=11)
tol=-0.000000001


def D(m,n):
    '''
    args:
        m,n: dimension of grid
    return:
        arr: (m+1)*(n+1) array, arr[i,j] is the deviation D of point (i,j), i.e, arr[i,j]=D(p,q)
    '''
    arr = np.zeros((m+1,n+1),dtype=np.float64)
    for i in range(m+1):
            for j in range(n+1):
                    arr[i,j]=abs(i/m-j/n)
    return arr

def get_map2(m,n,dis):
    '''
    args:
        m,n: dimension of grid
        dis: an arbitrary value of deviation
        
    return:
        maps: (m+1)*(n+1) array with value 0 or 1, all the points with Deviation D smaller or equal to dis are labeled as 1
    
    usage:
        If we want to find paths with deviation smaller or equal to dis, the path must only go through points labeled as 1
    '''
    maps=np.zeros((m+1,n+1))
    for i in range(m+1):
        for j in range(n+1):
            maps[i,j]=(dis-abs(i/m-j/n)>tol)
    return maps

def path2(maps):
    '''
    args:
        maps: (m+1)*(n+1) array with value 0 or 1, a returned matrix from function get_map2
    return:
        res: float number, a probability between 0 and 1, the probability of a possible path in the grid is in the given maps
    '''
    
    (m,n)=np.shape(maps)
    m-=1
    n-=1
    ref=np.zeros((m+1,n+1))-1

    def get_ref(maps,i,j):
        '''
        A function to implement dynamic programming algorithm
        '''
        if ref[i,j]>-1:
            return ref[i,j]

        if maps[i,j]==0:
            ref[i,j]=0
            return ref[i,j]
        
        if i==0 and j==0:
            ref[i,j]=1
            return ref[i,j]
        
        elif i==0:
            ref[i,j]=get_ref(maps,i,j-1)*0.5

        
        elif j==0:
            ref[i,j]=get_ref(maps,i-1,j)*0.5            
        
        else:
            ref[i,j]=get_ref(maps,i-1,j)*0.5+get_ref(maps,i,j-1)*0.5
            if i==m:
                ref[i,j]+=get_ref(maps,i,j-1)*0.5
            if j==n:
                ref[i,j]+=get_ref(maps,i-1,j)*0.5
        return ref[i,j]

    res=get_ref(maps,m,n)
    return res

def get_result(m,n):
    '''
    Define a function to print desired results
    '''
    uniq_D=np.array(sorted(list(set(np.reshape(D(m,n),(m+1)*(n+1))))))
    
    accum_prob=[]
    for d in uniq_D:
        maps=get_map2(m,n,d)
        prob=path2(maps)
        accum_prob.append(prob)
        
    probs=np.array(accum_prob)-np.array([0.0]+accum_prob[:-1])
    
    mean_D=np.sum(probs*uniq_D)
    sd_D=np.sum(probs*(uniq_D-mean_D)**2)**0.5
    conditional_prob=np.sum(probs*(uniq_D>0.6))/np.sum(probs*(uniq_D>0.2))
    
    print("Mean: {}".format(mean_D))
    print("Standard deviation: {}".format(sd_D))
    print("Conditional Probability: {}".format(conditional_prob))
    
    return (mean_D, sd_D, conditional_prob)
    
    

In [44]:
m=11
n=7
get_result(m,n)

Mean: 0.5188941460151176
Standard deviation: 0.1817855196893432
Conditional Probability: 0.3283139965440632


(0.51889414601511763, 0.1817855196893432, 0.32831399654406318)

In [46]:
m=23
n=31
get_result(m,n)

Mean: 0.3532883806846021
Standard deviation: 0.13694097230435626
Conditional Probability: 0.06312623037176399


(0.35328838068460211, 0.13694097230435626, 0.063126230371763986)

### Original version

In [2]:
import numpy as np

# define precision to ensure at least 10 digit precision
np.set_printoptions(precision=11)
tol=-0.000000001


def D(m,n):
    '''
    args:
        m,n: dimension of grid
    return:
        arr: (m+1)*(n+1) array, arr[i,j] is the deviation D of point (i,j), i.e, arr[i,j]=D(p,q)
    '''
    arr = np.zeros((m+1,n+1),dtype=np.float64)
    for i in range(m+1):
            for j in range(n+1):
                    arr[i,j]=abs(i/m-j/n)
    return arr

def get_map(m,n,p,q):
    '''
    args:
        m,n: dimension of grid
        p,q: a specific point (p,q) in the grid
        
    return:
        maps: (m+1)*(n+1) array with value 0 or 1, all the points with Deviation D smaller or equal to the D(p,q) are labeled as 1
        
    If we want to find paths with deviation smaller or equal to D(p,q), the path must go through only points labeled as 1
    '''
    dis=abs(p/m-q/n)
    maps=np.zeros((m+1,n+1))
    for i in range(m+1):
        for j in range(n+1):
            maps[i,j]=(dis-abs(i/m-j/n)>tol)
    return maps

def path(maps,p,q):
    '''
    args:
        maps: (m+1)*(n+1) array with value 0 or 1
        p,q: a specific point (p,q) in the grid
    return:
        float number, a probability between 0 and 1, the probability of a path in the grid 
        that is in the given maps and goes through point (p,q)
    '''
    (m,n)=np.shape(maps)
    m-=1
    n-=1
    ref=np.zeros((m+1,n+1))-1

    def get_ref(maps,i,j,a=0,b=0):
        '''
        Dynamic programming algorithm to calculate the probability of a path passing through point (i,j), given point(a,b) as starting point
        '''
        if ref[i,j]>-1:
            return ref[i,j]

        if maps[i,j]==0:
            ref[i,j]=0
            return ref[i,j]
        
        if i==0 and j==0:
            ref[i,j]=1
            return ref[i,j]
        
        if i==a and j==b:
            return ref[i,j]
        
        elif a==m:
            ref[i,j]=get_ref(maps,i,j-1,a,b)

        
        elif b==n:
            ref[i,j]=get_ref(maps,i-1,j,a,b)
                
        elif i==a:
            ref[i,j]=get_ref(maps,i,j-1,a,b)
            if i<m:
                ref[i,j]/=2
        
        elif j==b:
            ref[i,j]=get_ref(maps,i-1,j,a,b)
            if j<n:
                ref[i,j]/=2                
        
        else:
            ref[i,j]=get_ref(maps,i-1,j,a,b)/2+get_ref(maps,i,j-1,a,b)/2
            if i==m:
                ref[i,j]+=get_ref(maps,i,j-1,a,b)/2
            if j==n:
                ref[i,j]+=get_ref(maps,i-1,j,a,b)/2
        return ref[i,j]

    get_ref(maps,p,q,0,0)
    res=get_ref(maps,m,n,p,q)
    
    # Deduct double counted probability for paths crossing two points with same D 
    if p<(m/2) and q<(n/2):
        ref=np.zeros((m+1,n+1))-1
        get_ref(maps,p,q,0,0)
        get_ref(maps,m-p,n-q,p,q)
        minus=get_ref(maps,m,n,m-p,n-q)
        res-=minus
    return res
            
    
    

In [47]:
m=23
n=31
prob=np.zeros((m+1,n+1))
for p in range(m+1):
    for q in range(n+1):
        maps=get_map(m,n,p,q)
        prob[p,q]=path(maps,p,q)


In [48]:
# mean of D
mean_D=np.sum(D(m,n)*prob)
mean_D


0.35328838068460217

In [49]:
# standard diviation of D
variance=np.sum(((D(m,n)-mean_D)**2)*prob)
variance**0.5

0.13694097230435626

In [50]:
# conditional probability
Pr06=np.sum(prob*(D(m,n)>0.6))
Pr02=np.sum(prob*(D(m,n)>0.2))
Pr06/Pr02

0.063126230371763986